In [14]:
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from osgeo import gdal, ogr
import errno
import argparse
import itertools
import glob
from subprocess import call
from collections import defaultdict
import json
from datetime import timedelta
from datetime import datetime as dt
import shapely.wkt
from shapely.geometry import Polygon, box
from dateutil.relativedelta import relativedelta
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from multiprocessing import Pool, cpu_count
from itertools import repeat
import warnings

warnings.filterwarnings('ignore')

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
s3_client = boto3.client('s3')
gdal.SetConfigOption('AWS_NO_SIGN_REQUEST', 'YES')

import pandas as pd
from rasterstats import zonal_stats
import geopandas as gpd

In [15]:
def get_mgrs_shp(aoi_shp, bbox):
    aoi_mgrs_shp = 'sentinel2_bbox_mgrs.shp'
    try:
        os.remove(aoi_mgrs_shp)
    except OSError:
        pass
    if aoi_shp:
        call(str('ogr2ogr -clipsrc ' + aoi_shp + ' ' + aoi_mgrs_shp + ' ' + s2_tile), shell=True)
    else:
        min_x, min_y, max_x, max_y = bbox[0], bbox[1], bbox[2], bbox[3]
        call(str('ogr2ogr -f "ESRI Shapefile" ' + aoi_mgrs_shp + ' ' + s2_tile + ' -clipsrc ' + str(min_x)
                 + ' ' + str(min_y) + ' ' + str(max_x) + ' ' + str(max_y)), shell=True)
    return aoi_mgrs_shp


def shape_to_tiles(aoi_shp=None, bbox=None):
    aoi_mgrs_shp = get_mgrs_shp(aoi_shp, bbox)
    driver = ogr.GetDriverByName('ESRI Shapefile')
    ds = driver.Open(aoi_mgrs_shp)
    layer = ds.GetLayer(0)
    mgrs_list = []
    for feat in layer:
        mgrs_list.append(feat.GetField('tile'))
    del layer
    del ds
    return mgrs_list

def shape_to_polygon(shp_file=None, bbox=None):
    poly = None
    if bbox:
        poly = box(*bbox, ccw=True)
    elif shp_file:
        ds = ogr.Open(shp_file)
        layer = ds.GetLayer(0)
        for feat in layer:
            wkt_poly = feat.geometry().ExportToWkt()
        poly = shapely.wkt.loads(wkt_poly)
    return poly


def dates_dif(date1, date2):
    d0 = dt.strptime(date1, '%Y-%m-%d').date()
    d1 = dt.strptime(date2, '%Y-%m-%d').date()
    delta = abs(d1 - d0).days
    return delta


def validate_date(original_date, start_date, end_date):
    final_date = dt.strptime(str(original_date), '%Y-%m-%d')
    s_date = dt.strptime(str(start_date), '%Y-%m-%d')
    e_date = dt.strptime(str(end_date), '%Y-%m-%d')
    return s_date <= final_date <= e_date


def datetime_iterator(start_date=None, end_date=None):
    if not end_date:
        end_date = dt.today().date()
    if not start_date:
        start_date = end_date - timedelta(30)
    start_date = dt.strptime(str(start_date), '%Y-%m-%d').date()
    start_date = start_date.replace(day=1)
    end_date = dt.strptime(str(end_date), '%Y-%m-%d').date()
    while start_date <= end_date:
        yield start_date
        start_date = start_date + relativedelta(months=1)


def data_difference_days(product_dictionary, days_interval):
    from datetime import datetime as dt
    prev_date = None
    _dates = list(sorted(product_dictionary.keys()))
    n = len(_dates)
    for i in range(n - 1):
        if not prev_date:
            prev_date = _dates[i]
        date = _dates[i + 1]
        date_diff = dt.strptime(date, "%Y-%m-%d") - dt.strptime(prev_date, "%Y-%m-%d")
        days_diff = date_diff.days
        if days_diff < days_interval:
            product_dictionary.pop(date)
        else:
            prev_date = None
    return product_dictionary

In [16]:
def get_product_ids(start_date, end_date, cloud_threshold, data_days_interval, shape_file=None, bbox=None):
    source_s3_bucket = 'sentinel-cogs'
    source_s3_dir = 'sentinel-s2-l2a-cogs'
    tile_list = shape_to_tiles(aoi_shp=shape_file, bbox=bbox)
    tile_list = ['43RDM']
    poly = shape_to_polygon(shp_file=shape_file, bbox=bbox)
    print(f'Tiles found for the given AOI : {tile_list}')
    final_dict = {'single_tile': {}, 'merge_tile': {}}
    for tile in tile_list:
        utm_zone, lat_band, grid_square = str(tile)[:2], str(tile)[2], str(tile)[3:5]
        for _date in datetime_iterator(start_date, end_date):
            _year = int(_date.year)
            _month = int(_date.month)
            _PREFIX = os.path.join(source_s3_dir, str(utm_zone), str(lat_band), str(grid_square),
                                   str(_year), str(_month), '')
            _PREFIX = _PREFIX.replace('\\', '/')
            response = s3_client.list_objects(Bucket=source_s3_bucket, Prefix=_PREFIX)
            for content in response.get('Contents', []):
                key = content['Key']
                if key.endswith('.json'):
                    product_id = str(key).split('/')[-2]
                    pid_date = '-'.join([str(_year), str(_month).zfill(2), str(product_id[16:18]).zfill(2)])
                    if not validate_date(pid_date, start_date, end_date):
                        continue
                    result = s3.Object(source_s3_bucket, key)
                    data = json.load(result.get()['Body'])
                    tile_coord = data['geometry']['coordinates']
                    tile_cloud = data['properties']['eo:cloud_cover']
                    tile_poly = Polygon(tile_coord[0])
                    percent_area = poly.intersection(tile_poly).area / poly.area
                    print(f'Date: {pid_date}   Tile:{tile}   Cloud: {tile_cloud}   Area AOI/Tile: {percent_area}')
                    if percent_area <= 0.05:
                        continue
                    elif percent_area >= 0.99:
                        if not final_dict['single_tile'].get(str(pid_date)):
                            final_dict['single_tile'][str(pid_date)] = {}
                        final_dict['single_tile'][str(pid_date)][str('pids')] = [product_id]
                        final_dict['single_tile'][str(pid_date)][str('cloud_percentages')] = tile_cloud
                    else:
                        if not final_dict['merge_tile'].get(str(pid_date)):
                            final_dict['merge_tile'][str(pid_date)] = defaultdict(list)
                        final_dict['merge_tile'][str(pid_date)][str('pids')].append(product_id)
                        final_dict['merge_tile'][str(pid_date)][str('tile_ids')].append(tile)
                        final_dict['merge_tile'][str(pid_date)][str('cloud_percentages')].append(tile_cloud)
                        final_dict['merge_tile'][str(pid_date)][str('percent_areas')].append(percent_area)
    diff = dates_dif(start_date, end_date)
    diff = diff // data_days_interval
    diff = diff // 2  #### Data interval Buffer for user's input
    all_pids = {}
    if len(final_dict['single_tile']) >= diff:
        data = final_dict['single_tile']
        _dates = list(sorted(data.keys()))
        for date in _dates:
            print(f'Single Tile Metadata: {data[date]}')
            tile_cloud = data[date]['cloud_percentages']
            if tile_cloud > cloud_threshold:
                continue
            else:
                all_pids[str(date)] = data[date]['pids']
    else:
        data = final_dict['merge_tile']
        prev_date = None
        skip_one = False
        _dates = list(sorted(data.keys()))
        for date in _dates:
            if not prev_date:
                prev_date = date
                continue
            if skip_one:
                prev_date = date
                skip_one = False
                continue
            date_diff = dt.strptime(date, "%Y-%m-%d") - dt.strptime(prev_date, "%Y-%m-%d")
            days_diff = date_diff.days
            weighted_sum = 0
            sum_area_percents = 0
            for i in range(len(data[prev_date]['tile_ids'])):
                weighted_sum += data[prev_date]['cloud_percentages'][i] * data[prev_date]['percent_areas'][i]
                sum_area_percents += data[prev_date]['percent_areas'][i]

            for i in range(len(data[date]['tile_ids'])):
                weighted_sum += data[date]['cloud_percentages'][i] * data[date]['percent_areas'][i]
                sum_area_percents += data[date]['percent_areas'][i]

            avg_cloud_percent = weighted_sum / sum_area_percents
            if avg_cloud_percent > cloud_threshold:
                prev_date = date
                continue
            if days_diff >= 5:
                all_pids[str(prev_date)] = data[prev_date]['pids']
            elif days_diff < 5:
                all_pids[str(date)] = data[prev_date]['pids'] + data[date]['pids']
                skip_one = True
            prev_date = date
    final_pids = data_difference_days(all_pids, data_days_interval)
    return final_pids

In [17]:
source_s3_bucket = 'sentinel-cogs'
source_s3_dir = 'sentinel-s2-l2a-cogs'
def pid_to_path(prod_id, band):
    lst = prod_id.split('_')
    tile = lst[1]
    utm_zone, lat_band, grid_sq = str(tile)[:2], str(tile)[2], str(tile)[3:5]
    _year, _month, _day = lst[2][0:4], lst[2][4:6], lst[2][6:8]
    vsi_ext = '/vsis3/'
    band_tif = os.path.join(vsi_ext, source_s3_bucket, source_s3_dir, str(utm_zone), str(lat_band),
                            str(grid_sq),str(_year), str(int(_month)), str(prod_id), str(band) + '.tif')
    band_tif = band_tif.replace('\\', '/')
    return band_tif

In [18]:
def raster_to_array(raster_file):
    ds = gdal.Open(raster_file)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    return arr
def get_ndvi(red, nir):
    arr = (nir - red) / (nir + red)
    arr[arr > 1] = 0
    arr[arr < -1] = 0
    return arr
def get_savi(red, nir, l): ## l = soil brightness correction factor could range from (0 -1)
    arr = (1.0 + l)*(nir - red) / (nir + red + l)
    return arr

def get_lswi(narrow_nir, swir):
    arr = (narrow_nir - swir) / (narrow_nir + swir)
    arr[arr > 1] = 0
    arr[arr < -1] = 0
    return arr

In [22]:
def clip_raster(raster_file, output_file=None, shp_file=None, bbox=None, out_width=None, out_height=None,
                      return_file=True):
#     ds_lst = list()
    ds = gdal.BuildVRT('', raster_file, VRTNodata=0, srcNodata=0)
    ds1 = gdal.Warp(output_file, ds, format='GTiff', dstNodata=0,
              dstSRS="+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0",
              cutlineDSName=shp_file, cropToCutline=True)
    arr = ds1.ReadAsArray()
    arr = arr.astype('int16')

def write_raster(ref_raster, array, dst_filename, gdal_GDT_datatype):
    driver = gdal.GetDriverByName('GTiff')
    cols = array.shape[1]
    rows = array.shape[0]
    out_ds = driver.Create(dst_filename, cols, rows, 1, gdal_GDT_datatype)
    out_ds.GetRasterBand(1).WriteArray(array)

    # adding GeoTransform and Projection
    data0 = gdal.Open(ref_raster)
    geo_trans = data0.GetGeoTransform()
    proj = data0.GetProjection()
    del data0
    out_ds.SetGeoTransform(geo_trans)
    out_ds.SetProjection(proj)
    out_ds.FlushCache()
    del out_ds
    return dst_filename

In [21]:
# s2_tile = '/home/ec2-user/SageMaker/sumit/crop-classification/unsupcc/satellite/satellite_tiles/s2_tile.shp'
# shape_file = 'data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp'
# pids = get_product_ids('2021-10-01', '2022-03-30', 10, 5, 
#                        shape_file=shape_file, bbox=None)

In [6]:
for key, val in pids.items():
    pid = val[0]
    date = pid.split('_')[2]
    f_path = os.path.join('data', 'indice', str(date))
    try:
        os.makedirs(f_path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass
    b4_path = pid_to_path(pid, 'B04')
    b8_path = pid_to_path(pid, 'B08')
    b8a_path = pid_to_path(pid, 'B8A')
    b11_path = pid_to_path(pid, 'B11')
    
    b4 = clip_raster(raster_file=b4_path, output_file='b4.tif', shp_file='data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp')
    b8 = clip_raster(raster_file=b8_path, output_file='b8.tif', shp_file='data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp')
    
    ndvi = get_ndvi(b4,b8)
    ndvi = np.around(ndvi, decimals=2, out=None)
    write_raster('b4.tif', ndvi, f'{f_path}/ndvi.tif', gdal.GDT_Float32)
    
    savi = get_savi(b4, b8, 0.428)
    savi = np.around(savi, decimals=2, out=None)
    write_raster('b4.tif', savi, f'{f_path}/savi.tif', gdal.GDT_Float32)
    b4 = b8 = ndvi = savi = None
    
    b8a = clip_raster(raster_file=b8a_path, output_file='b8a.tif', shp_file='data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp')
    b11 = clip_raster(raster_file=b11_path, output_file='b11.tif', shp_file='data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp')
    
    lswi = get_lswi(b8a, b11)
    lswi = np.around(lswi, decimals=2, out=None)
    write_raster('b8a.tif', lswi, f'{f_path}/lswi.tif', gdal.GDT_Float32)

NameError: name 'pids' is not defined

In [ ]:
import random
geodf = gpd.read_file("data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp")
date_dir = glob.glob('data/indice/*')
date_dir = sorted(date_dir)
randomlist = random.sample(range(0, len(geodf)), 10)
print(randomlist)
for i in randomlist:
    dd = defaultdict(list)
    f_path = f'indice_ts_data'
    try:
        os.makedirs(f_path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass
    for _date in date_dir:
        data_date = str(_date.split('/')[-1])
        dd['date'].append(data_date)
        for indice in ['ndvi', 'savi', 'lswi']:
            lst = zonal_stats(geodf[i:i+1], f'{_date}/{indice}.tif', stats="mean majority")
            dd[f'{indice}_mean'].append(lst[0]['mean'])
            dd[f'{indice}_majority'].append(lst[0]['majority'])
    df = pd.DataFrame(dd)
    df['Khasra_no'] = geodf['Khasra_no'].iloc[i]
    df['geometry'] = geodf['geometry'].iloc[i]
    df = df.round(3)
    df = df[['Khasra_no', 'date', 'ndvi_mean', 'ndvi_majority', 'savi_mean', 'savi_majority','lswi_mean', 'lswi_majority', 'geometry']]
    df.to_csv(f_path + f'/farm{i}.csv', index=False)
#     gdf = gpd.GeoDataFrame(df, geometry=geometry)
#     gdf.to_file(f_path + f'/farm{i}.shp')

In [ ]:
drs = glob.glob('indice_ts_data/*')
for item in drs:
    df = pd.read_csv(item)
    print(item, df.head(3))

In [ ]:
i = 1
import rasterio
from rasterio.mask import mask
import geopandas as gpd
shapefile = gpd.read_file("data/farm_data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp")
# extract the geometry in GeoJSON format
geoms = shapefile.geometry.values # list of shapely geometries
geometry = geoms[i] # shapely geometry
# transform to GeJSON format
from shapely.geometry import mapping
geoms = [mapping(geoms[i])]
# extract the raster values values within the polygon 
with rasterio.open("data/indice/20211218/ndvi.tif") as src:
     out_image, out_transform = mask(src, geoms, crop=True)

In [ ]:
arr = out_image[0]

import matplotlib.pyplot as plt 
plt.imshow(arr)